# Mô hình phân loại email/sms rác (spam classification) sử dụng machine learning. Quy trình theo chuẩn NLP (Natural Language Processing)

# CAC QUY TRÌNH
## 1. Tiền xử lý dữ liệu: Đọc dữ liệu CSV, làm sạch văn bản (bỏ URL, email, ký tự thừa, chuyển lowercase), cắt từ (tokenize), loại bỏ từ vô nghĩa (stopwords), và chuẩn hóa từ (lemmatization). Mục đích: làm sạch dữ liệu, giúp mô hình học hiểu quả hơn (tăng accuracy 10-20%)

## 2. Vector hóa (Feature Extraction): Chuyển văn bản thành số (sử dụng TF-IDF) để mô hình có thể xử lú. Mục đích: Văn bản không phải số, nên cần biểu diễn dưới dạng vector để tính toán

## 3. Chuẩn bị nhãn và huấn luyện mô hình: Map nhãn "ham/spam" (ham là không vi phạm/ spam là vi phạm) thành 0/1, chia train/test, huấn luyện Neive Bayes (hoặc Logistic Regresion). Mục đích: Học từ dữ liệu để dự đoán "spam/ham", đánh giá accuracy/recall.

## 4. Dự đoán và đánh giá: Dự đoán trên dữ liệu mới. Mục đích: Kiểm tra mô hình thực tế.


# Thực hành

## 1. Đọc dữ liệu từ file spam.csv vào DataFrame để chuẩn bị xử lý

In [84]:
import pandas as pd
train_model=pd.read_csv("dataset/spam.csv")
print(train_model.head())

  Label                                               Text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


## 2.Chuyển hóa dữ liệu chữ hoa về thành chữ thường. (Khi phân loại 'Hello' và 'hello' được coi là giống nhau)

In [85]:
train_model["Text"]=train_model["Text"].str.lower()

### 3. Xây dựng hàm để làm sạch dữ liệu Text

In [87]:
import re
import string

def clean_text(text):
    if pd.isnull(text):  # Xử lý null
        return ""
    text = text.lower()  # Kết hợp lowercase
    text = re.sub(r"http\S+|www\S+", " ", text)  # Bỏ URL
    text = re.sub(r"\S+@\S+", " ", text)         # Bỏ email
    text = re.sub(r"[^a-z\s]", " ", text)        # Bỏ số, ký tự (giữ a-z và khoảng trắng)
    text = text.translate(str.maketrans('', '', string.punctuation))  # Bỏ punctuation
    text = re.sub(r"\s+", " ", text).strip()     # Bỏ khoảng trắng thừa
    return text

train_model['text_clean'] = train_model["Text"].apply(clean_text) # thêm mới cột clean này.


### Mục đích: văn bản chứa nhiều thông tin thừa -> làm sạch để giữ lại phần từ ngữ có nghĩa. Việc này giúp giảm nhiễu, tăng hiệu quả mô hình.


In [88]:
print(train_model.columns)
print("Trước clean:\n", train_model['Text'].head())
print("Sau clean:\n", train_model['text_clean'].head())

Index(['Label', 'Text', 'text_clean'], dtype='object')
Trước clean:
 0    go until jurong point, crazy.. available only ...
1                        ok lar... joking wif u oni...
2    free entry in 2 a wkly comp to win fa cup fina...
3    u dun say so early hor... u c already then say...
4    nah i don't think he goes to usf, he lives aro...
Name: Text, dtype: object
Sau clean:
 0    go until jurong point crazy available only in ...
1                              ok lar joking wif u oni
2    free entry in a wkly comp to win fa cup final ...
3          u dun say so early hor u c already then say
4    nah i don t think he goes to usf he lives arou...
Name: text_clean, dtype: object


In [71]:
import nltk
nltk.download('punkt')   # tải tokenizer
print(nltk.data.path)

['C:\\Users\\ADMIN/nltk_data', 'c:\\ProgramData\\anaconda3\\nltk_data', 'c:\\ProgramData\\anaconda3\\share\\nltk_data', 'c:\\ProgramData\\anaconda3\\lib\\nltk_data', 'C:\\Users\\ADMIN\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [89]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ssl  # Fix SSL nếu cần

# Fix SSL cho Windows/Anaconda (nếu download fail)
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Download đầy đủ (force nếu cần)
nltk.download('punkt', force=False)  # Giữ cũ nếu có
nltk.download('punkt_tab')  # Key: Download tokenizer mới
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Cho lemmatizer đầy đủ

# Kiểm tra version và path (debug)
print("NLTK version:", nltk.__version__)
print("NLTK data paths:", nltk.data.path)

stop_words = set(stopwords.words('english')) #tạo tập hợp để lọc từ vô nghĩa.
lemmatizer = WordNetLemmatizer() # để chuẩn hóa từ

def preprocess_text(text):
    if pd.isnull(text):
        return [] # Nếu text rỗng thì trả về list rỗng
    try:
        # Thử word_tokenize với punkt_tab
        tokens = word_tokenize(text.lower())  # Lowercase ở đây
        filtered_tokens = []
        for w in tokens:
            if w not in stop_words and len(w) > 2 and w.isalpha():  # Bỏ từ ngắn, không chữ cái
                try:
                    lemma = lemmatizer.lemmatize(w)
                    filtered_tokens.append(lemma)
                except Exception:
                    filtered_tokens.append(w)
        return filtered_tokens
    except LookupError as e:  # Nếu vẫn lỗi punkt_tab
        print(f"Tokenizer error: {e}. Fallback to split().")
        # Fallback: split đơn giản (không cần punkt)
        tokens = text.lower().split()
        filtered_tokens = [lemmatizer.lemmatize(w) for w in tokens 
                           if w not in stop_words and len(w) > 2 and w.isalpha()]
        return filtered_tokens
    except Exception as e:
        print(f"Lỗi khác: {e}")
        return []

# Áp dụng (sẽ dùng fallback nếu cần, chạy nhanh)
print("Đang preprocess tokens...")
train_model['tokens'] = train_model['text_clean'].fillna('').apply(preprocess_text)
print(train_model[['text_clean', 'tokens']].head())
print("Ví dụ tokens dòng 0:", train_model['tokens'].iloc[0][:10])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


NLTK version: 3.9.1
NLTK data paths: ['C:\\Users\\ADMIN/nltk_data', 'c:\\ProgramData\\anaconda3\\nltk_data', 'c:\\ProgramData\\anaconda3\\share\\nltk_data', 'c:\\ProgramData\\anaconda3\\lib\\nltk_data', 'C:\\Users\\ADMIN\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']
Đang preprocess tokens...
                                          text_clean  \
0  go until jurong point crazy available only in ...   
1                            ok lar joking wif u oni   
2  free entry in a wkly comp to win fa cup final ...   
3        u dun say so early hor u c already then say   
4  nah i don t think he goes to usf he lives arou...   

                                              tokens  
0  [jurong, point, crazy, available, bugis, great...  
1                            [lar, joking, wif, oni]  
2  [free, entry, wkly, comp, win, cup, final, tkt...  
3               [dun, say, early, hor, already, say]  
4        [nah, think, go, usf, life, around, though]  
Ví d

## Thêm cột tokens vào train

In [91]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

train_model['tokens'] = train_model['tokens'].apply(lambda x: [w for w in x if w not in stop_words])
print(train_model.head())


  Label                                               Text  \
0   ham  go until jurong point, crazy.. available only ...   
1   ham                      ok lar... joking wif u oni...   
2  spam  free entry in 2 a wkly comp to win fa cup fina...   
3   ham  u dun say so early hor... u c already then say...   
4   ham  nah i don't think he goes to usf, he lives aro...   

                                          text_clean  \
0  go until jurong point crazy available only in ...   
1                            ok lar joking wif u oni   
2  free entry in a wkly comp to win fa cup final ...   
3        u dun say so early hor u c already then say   
4  nah i don t think he goes to usf he lives arou...   

                                              tokens  
0  [jurong, point, crazy, available, bugis, great...  
1                            [lar, joking, wif, oni]  
2  [free, entry, wkly, comp, win, cup, final, tkt...  
3               [dun, say, early, hor, already, say]  
4        [nah, 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## TfidfVectorizer không chấp nhận danh sách token trực tiếp; nó cần chuỗi văn bản để tự động tách từ (tokenize). Việc nối token thành chuỗi đảm bảo dữ liệu tương thích với vectorizer. Tư duy: Chuẩn bị dữ liệu đúng định dạng để tránh lỗi và tận dụng tối đa chức năng của thư viện.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF cần chuỗi, nên nối lại tokens thành 1 string
train_model['text_final'] = train_model['tokens'].apply(lambda x: ' '.join(x))

vectorizer = TfidfVectorizer() # khởi tạo đối tượng
X = vectorizer.fit_transform(train_model['text_final']) # học từ đoeẻm từ các văn bản trong text_final
# chuyển đổi văn bản thành một vecctor TF-IDF tạo ra ma trận thưa X

print(X.shape)  # số dòng x số từ (emails x vocab)
# số dòng = số văn bản trong train_model[text_final] (số mail)
# số cột = số từ duy nhất trong từ điển (vocab size) do TfidfVectorizer


(5572, 6800)


In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Khởi tạo TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # chọn 5000 đặc trưng quan trọng nhất

# Fit và transform train data
X = tfidf.fit_transform(train_model['text_clean'])  # text_clean là cột đã xử lý văn bản

print("Kích thước ma trận TF-IDF:", X.shape)


Kích thước ma trận TF-IDF: (5572, 5000)


In [76]:
# Giả sử cột nhãn gốc tên là 'label'
# spam = 1, ham = 0
train_model['label_num'] = train_model['Label'].map({'ham': 0, 'spam': 1})

print(train_model[['Label', 'label_num']].head())


  Label  label_num
0   ham          0
1   ham          0
2  spam          1
3   ham          0
4   ham          0


In [77]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Y là nhãn spam/ham
y = train_model['label_num']  # giả sử bạn đã đổi 'spam' = 1, 'ham' = 0

# Chia dữ liệu train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo model Naive Bayes
model = MultinomialNB()

# Train
model.fit(X_train, y_train)

# Dự đoán
y_pred = model.predict(X_test)

# Đánh giá
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nBáo cáo phân loại:\n", classification_report(y_test, y_pred))


Accuracy: 0.9650224215246637

Báo cáo phân loại:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       966
           1       1.00      0.74      0.85       149

    accuracy                           0.97      1115
   macro avg       0.98      0.87      0.91      1115
weighted avg       0.97      0.97      0.96      1115



In [94]:
sample = ["Thanks for your subscription to Ringtone UK your mobile will be charged £5/month Please confirm by replying YES or NO. If you reply NO you will not be charged"]

sample_vec = tfidf.transform(sample)
print("Dự đoán:", model.predict(sample_vec))  # 1 = spam, 0 = ham


Dự đoán: [0]
